# IMDb 감성 분석: 임베딩 활용

이 노트북은 IMDb 영화 리뷰 데이터셋을 사용하여 감성 분석 모델을 구축하고 훈련하는 과정을 안내합니다. 자연어 처리(NLP)의 핵심 단계인 데이터 다운로드, 전처리, 텍스트 벡터화, 그리고 Keras를 사용한 양방향 LSTM 모델 구축 및 평가를 다룹.

## 1. 라이브러리 임포트

프로젝트에 필요한 모든 라이브러리를 임포트합니다.

In [ ]:
import requests
import subprocess
import os
import pathlib
import shutil
import random
import tensorflow as tf
import keras
from keras import models, layers
from tensorflow.keras.layers import TextVectorization

## 2. 데이터 준비

IMDb 데이터셋을 다운로드하고 압축을 해제한 후, 훈련(train) 및 검증(validation) 세트로 구성합니다.

1.  **`download()`**: Stanford AI 그룹의 서버에서 `aclImdb_v1.tar.gz` 파일을 다운로드합니다.
2.  **`release()`**: `tar` 명령을 사용하여 다운로드한 파일의 압축을 해제합니다.
3.  **`labeling()`**: 기존 `train` 디렉토리의 데이터 중 20%를 `val` 디렉토리로 옮겨 검증 세트를 생성합니다. 이 과정은 데이터가 없을 경우에만 한 번 실행하면 됩니다.

In [ ]:
def download():
    """지정된 URL에서 aclImdb_v1.tar.gz 파일을 다운로드합니다."""
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"
    
    if not os.path.exists(file_name):
        print(f"{file_name} 다운로드를 시작합니다...")
        response = requests.get(url, stream=True)
        with open(file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print("다운로드 완료!")
    else:
        print(f"{file_name}이(가) 이미 존재합니다.")

def release():
    """tar.gz 파일의 압축을 해제합니다."""
    if not os.path.exists("aclImdb"):
        print("압축 해제를 시작합니다...")
        # Windows에서는 tar 명령어 사용을 위해 shell=True가 필요할 수 있습니다.
        subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True)
        print("압축 해제 완료!")
        # 불필요한 unsup 디렉토리 삭제
        if os.path.exists("aclImdb/train/unsup"):
            shutil.rmtree("aclImdb/train/unsup")
            print("불필요한 unsup 디렉토리를 삭제했습니다.")
    else:
        print("aclImdb 디렉토리가 이미 존재합니다.")

def labeling():
    """훈련 세트의 일부를 검증 세트로 분리합니다."""
    base_dir = pathlib.Path("aclImdb")
    val_dir = base_dir / "val"
    train_dir = base_dir / "train"
    
    if not val_dir.exists():
        print("검증 세트 생성을 시작합니다...")
        for category in ("neg", "pos"):
            os.makedirs(val_dir / category)
            files = os.listdir(train_dir / category)
            random.Random(1337).shuffle(files) # 재현 가능성을 위해 시드 설정
            num_val_samples = int(0.2 * len(files))
            val_files = files[-num_val_samples:]
            for fname in val_files:
                shutil.move(train_dir / category / fname, val_dir / category / fname)
        print("검증 세트 생성 완료!")
    else:
        print("val 디렉토리가 이미 존재합니다.")

# 데이터 준비 함수 실행
download()
release()
labeling()

## 3. 데이터셋 로드

`text_dataset_from_directory` 유틸리티를 사용하여 디렉토리 구조를 기반으로 훈련, 검증, 테스트 데이터셋을 생성합니다. 이 함수는 자동으로 텍스트 파일과 해당 라벨(디렉토리 이름 기반)을 `tf.data.Dataset` 객체로 로드합니다.

In [ ]:
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

### 데이터셋 구조 확인

데이터셋의 한 배치를 샘플링하여 입력(텍스트)과 타겟(라벨)의 형태와 데이터 타입을 확인합니다.

In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

## 4. 텍스트 벡터화

신경망이 텍스트를 처리할 수 있도록, 각 단어를 정수 인덱스로 변환하는 과정이 필요합니다. `TextVectorization` 층을 사용하여 이 과정을 수행합니다.

- `max_tokens`: 어휘 사전에 포함할 최대 단어 수 (빈도수 기준).
- `output_sequence_length`: 모든 시퀀스를 동일한 길이로 만들기 위한 최대 길이. 이보다 길면 잘리고, 짧으면 0으로 채워집니다.

In [ ]:
max_length = 600
max_tokens = 20000

text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)

# 라벨을 제외한 텍스트 데이터셋만 사용하여 어휘 사전을 구축합니다.
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

### 데이터셋에 벡터화 적용

구축된 어휘 사전을 사용하여 훈련, 검증, 테스트 데이터셋의 텍스트를 정수 시퀀스로 변환합니다.

In [ ]:
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE)

## 5. 모델 구축

이제 감성 분석을 위한 모델을 정의합니다. 이 모델은 다음 층으로 구성됩니다.

1.  **`Input`**: 정수 시퀀스를 입력으로 받습니다.
2.  **`Embedding`**: 각 정수 인덱스를 고정된 크기의 밀집 벡터로 변환합니다. 이 층은 단어 간의 의미적 관계를 학습합니다.
3.  **`Bidirectional(LSTM)`**: 순방향과 역방향으로 시퀀스를 처리하여 문맥을 더 잘 파악하는 양방향 RNN 층입니다.
4.  **`Dropout`**: 과적합을 방지하기 위해 일부 뉴런을 무작위로 비활성화합니다.
5.  **`Dense`**: 최종적으로 긍정 또는 부정 클래스를 예측하는 출력 층입니다 (이진 분류이므로 `sigmoid` 활성화 함수 사용).

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")

# Embedding 층: input_dim은 어휘 사전 크기, output_dim은 임베딩 벡터의 차원
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)

# 양방향 LSTM
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)

# 출력층
outputs = layers.Dense(1, activation='sigmoid')(x)

# 모델 생성
model = keras.Model(inputs, outputs)

# 모델 컴파일
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# 모델 구조 요약
model.summary()

## 6. 모델 훈련

준비된 훈련 데이터셋으로 모델을 훈련시키고, 매 에포크마다 검증 데이터셋으로 성능을 평가합니다.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidirectional_lstm.keras",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

## 7. 모델 평가

훈련이 완료된 후, 한 번도 보지 않은 테스트 데이터셋을 사용하여 모델의 최종 성능을 평가합니다.

In [ ]:
print("\n테스트셋 평가 결과:")
model.evaluate(int_test_ds)